In [1]:
!pip install torch transformers datasets trl unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 

In [2]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048, load_in_4bit=True,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model, r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
)

# Set up chat template and prepare dataset
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
dataset = load_dataset("mlabonne/FineTome-100k", split="train")
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(
    lambda examples: {
        "text": [
            tokenizer.apply_chat_template(convo, tokenize=False)
            for convo in examples["conversations"]
        ]
    },
    batched=True
)

# Set up trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir="outputs",
    ),
)

# Train the model
trainer.train()



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: soureesh1211 (soureesh1211-university-of-illinois-chicago) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.499200
2,2.639700
3,1.633800
4,1.952200
5,1.909300
6,1.985500
7,1.164900
8,1.865700
9,1.518800
10,1.458600


In [3]:
model.save_pretrained("finetuned_llama3")
tokenizer.save_pretrained("finetuned_llama3")

('finetuned_llama3/tokenizer_config.json',
 'finetuned_llama3/special_tokens_map.json',
 'finetuned_llama3/tokenizer.json')

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the base model with reduced memory footprint
base_model_id = "unsloth/Llama-3.2-3B-Instruct"  # Change if different
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,  # Reduced precision to save memory
    device_map="auto"  # Offload to GPU if available
)

# Load the LoRA adapter
adapter_path = "finetuned_llama3"  # Your fine-tuned LoRA adapter folder
model = PeftModel.from_pretrained(model, adapter_path)

# Merge LoRA into the base model
print("Merging LoRA adapter with the base model...")
model = model.merge_and_unload()

# Save the merged model efficiently
save_path = "merged_llama3"
model.save_pretrained(save_path, max_shard_size="2GB")  # Save in shards to avoid RAM overflow
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.save_pretrained(save_path)

print("✅ Model successfully merged and saved!")


Merging LoRA adapter with the base model...


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

✅ Model successfully merged and saved!


In [4]:
from huggingface_hub import login, upload_folder

# Log in to your Hugging Face account
login()


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
import os
os.environ["HF_TOKEN"] = ""

from huggingface_hub import upload_folder
upload_folder(
    folder_path="merged_llama3",
    repo_id="soureesh1211/finetuned-llama3",
    repo_type="model",
    token=os.environ["HF_TOKEN"]
)


model-00002-of-00004.safetensors:   0%|          | 0.00/1.96G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/1.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/503M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/soureesh1211/finetuned-llama3/commit/0d53660f63fe8241b8987708aeaac24116b1a758', commit_message='Upload folder using huggingface_hub', commit_description='', oid='0d53660f63fe8241b8987708aeaac24116b1a758', pr_url=None, repo_url=RepoUrl('https://huggingface.co/soureesh1211/finetuned-llama3', endpoint='https://huggingface.co', repo_type='model', repo_id='soureesh1211/finetuned-llama3'), pr_revision=None, pr_num=None)